In [64]:
# Import Library
import pandas as pd

In [65]:
# Import Datasets
airQualityData = pd.read_csv('global air pollution dataset.csv')
cityPopulationData = pd.read_csv('World Largest Cities by Population 2024.csv')

In [66]:
# Drop Columns
airQualityData.drop(columns=['AQI Category', 'CO AQI Value', 'CO AQI Category', 'Ozone AQI Value', 'Ozone AQI Category',
                             'NO2 AQI Value', 'NO2 AQI Category', 'PM2.5 AQI Value', 'PM2.5 AQI Category'], inplace=True)
airQualityData.head()

,Country,City,AQI Value
0,Russian Federation,Praskoveya,51
1,Brazil,Presidente Dutra,41
2,Italy,Priolo Gargallo,66
3,Poland,Przasnysz,34
4,France,Punaauia,22


In [67]:
# Drop Columns
cityPopulationData.drop(columns=['Unnamed: 0', 'Population (2023)', 'Growth Rate'], inplace=True)
cityPopulationData.head()

,City,Country,Population (2024)
0,Tokyo,Japan,37115035
1,Delhi,India,33807403
2,Shanghai,China,29867918
3,Dhaka,Bangladesh,23935652
4,Sao Paulo,Brazil,22806704


In [68]:
# Merge both of datasets
mergedData = pd.merge(airQualityData, cityPopulationData, on=['City', 'Country'], how='inner')

# Remove missing values in columns
mergedData.dropna(subset=['AQI Value', 'Population (2024)'], inplace=True)

# Rename the features
mergedData.rename(columns={'Country' : 'country_name'}, inplace=True)
mergedData.rename(columns={'City' : 'city_name'}, inplace=True)
mergedData.rename(columns={'AQI Value' : 'aqi_value'}, inplace=True)
mergedData.rename(columns={'Population (2024)' : 'population_2024'}, inplace=True)
mergedData.head()

,country_name,city_name,aqi_value,population_2024
0,South Africa,Port Elizabeth,79,1312631
1,China,Hangzhou,203,8419842
2,China,Tianjin,142,14470873
3,India,Jabalpur,170,1551004
4,China,Yanji,69,775672


In [69]:
print("Length of 'mergedData' =", len(mergedData), "rows")

Length of 'mergedData' = 473 rows


In [70]:
# Aggregate data based on 'country_name' & 'city_name', and sum all the 'population_2024'
mergedData = mergedData.groupby(['country_name', 'city_name'], as_index=False).agg({'aqi_value': 'first', 'population_2024': 'sum'})
print("Length of 'mergedData' =", len(mergedData), "rows")

Length of 'mergedData' = 472 rows


In [71]:
# Sort 'mergedData' by 'population_2024'
mergedData = mergedData.sort_values(by='population_2024', ascending=False)
mergedData.head(472)

,country_name,city_name,aqi_value,population_2024
324,Japan,Tokyo,79,37115035
254,India,Delhi,500,33807403
149,China,Shanghai,156,29867918
43,Brazil,Sao Paulo,198,22806704
214,Egypt,Cairo,81,22623874
...,...,...,...,...
82,China,Dongtai,159,756344
248,India,Bhavnagar,103,751493
4,Angola,Huambo,28,751297
39,Brazil,Ribeirao Preto,100,750174


In [72]:
# Classify population category based on 'population_2024'
def classify_population(population):
    if population <= 1000000:
        return 'Small'
    elif population <= 5000000:
        return 'Medium'
    elif population <= 10000000:
        return 'Large'
    elif population <= 20000000:
        return 'Very Large'
    elif population > 20000000:
        return 'Giant'
    
mergedData['population_category'] = mergedData['population_2024'].apply(classify_population)
mergedData.head(10)

,country_name,city_name,aqi_value,population_2024,population_category
324,Japan,Tokyo,79,37115035,Giant
254,India,Delhi,500,33807403,Giant
149,China,Shanghai,156,29867918,Giant
43,Brazil,Sao Paulo,198,22806704,Giant
214,Egypt,Cairo,81,22623874,Giant
75,China,Chongqing,283,17773923,Very Large
400,Pakistan,Karachi,151,17648555,Very Large
454,Turkey,Istanbul,59,16047350,Very Large
415,Philippines,Manila,131,14941953,Very Large
89,China,Guangzhou,184,14590096,Very Large


In [73]:
mergedData.to_csv('global_AQI_and_population_dataset.csv', index=False)